## 事前処理

In [1]:
from keras.datasets import mnist
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
X_train.shape

(60000, 28, 28)

In [4]:
X_test.shape

(10000, 28, 28)

In [0]:
X_train = X_train.reshape(60000, 784).astype('float32')
X_test = X_test.reshape(10000, 784).astype('float32')

In [6]:
X_train.shape

(60000, 784)

In [0]:
#[0,255]の値を[0,1]に正規化
X_train /= 255.0
X_test /= 255.0

In [8]:
y_train.shape

(60000,)

In [0]:
# 1 of Kのベクトルに変換
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [10]:
y_train.shape

(60000, 10)

![代替テキスト](https://lh3.googleusercontent.com/-YdRek8As5BA/WjYP-HqxsQI/AAAAAAAAD7g/RO0nHSWaBSEVljvJu08Q9qveAzbp4S5KACE0YBhgL/s1024/mnist%25E3%2580%2580%25E5%2585%25A5%25E5%258A%259B.png)

入力のイメージは下図。MNISTは28×28の図だが、784次元の1次元ベクトルに変換してから入力している

## モデル作成

**model = Sequential()で初期化**

**model.add()で層を積んでいく**

・　一層目:IN-784次元, OUT-64次元, 活性化関数-Selu

・　二層目:IN-default(多分64次元) OUT-10次元 活性化関数-SoftMax関数

**・デフォである活性化関数**：softplus、softsign、relu、tanh、sigmoid、hard_sigmoid、linear、elu (Keras2.xより追加)、selu (Keras2.xより追加)

**・より高度な活性化関数** ：PReLU、LearkyReLU

In [0]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(activation='selu', units=64, input_dim=784))
model.add(Dense(activation='softmax', units=10))

## 学習プロセス

**model.compile**

　・損失関数 : どのくらいの誤差があるかを定量化
 
　・最適化関数 : 誤差からのパラメータ更新の仕方

**loss : 損失関数**

![代替テキスト](https://drive.google.com/file/d/1G5fE-RKixkGm4lPSao5K4JKZwct3uc5Z/view?usp=sharing)

・mean_squared_error, mse:	二乗誤差

・mean_absolute_error, mae:	絶対誤差

・mean_absolute_percentage_error, make:	正解とのズレ(絶対値)の割合

・mean_squared_logarithmic_error, msle:	正解とのズレ(二乗誤差)の割合

・squared_hinge:	マイナスのところは0, プラスは二乗誤差

・hinge:	マイナスのところは0, プラスは絶対値

・binary_crossentropy	logloss

・categorical_crossentropy:	マルチクラスloglossとして知られている(ラベルがバイナリ配列であることが必要)

・sparse_categorical_crossentropy:	スパースラベルを取る

・kullback_leibler_divergence, kld:	確率を分布とみなした時の差

・poisson:	予測-正解*log(予測)の平均

・cosine_proximity:	予測と正解間のコサイン近似の負の平均

**optimizer : 最適化**

デフォで用意されている関数：

・SGD

・RMSprop(デフォルトパラメータ推奨)

・Adagrad(デフォルトパラメータ推奨)

・Adadelta(デフォルトパラメータ推奨)

・Adam(デフォルトは提案論文通り)

・Adamax Adamの拡張(無限ノルム)

・Nadam Adam+RMSprop with momentum

・TFOptimizer 

参考：http://www.procrasist.com/entry/2017/01/07/154441

In [0]:
from keras.optimizers import SGD
# optimizer = "sgd"でも、簡単に最適化関数の設定ができる
#(その場合パラメータはデフォルト値)

model.compile(loss='categorical_crossentropy',
             optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True),
             metrics=['accuracy'])

### モデルの学習

**model.fit()でモデルを学習**

引数は以下

・**X_train** : データ(MNISTだと、どこに何個データが入っているか)

・**y_train **: ラベル(1-10)

・**batch_size** : ミニバッチにデータを分けるこの場合60000個ある学習データを、32個ずつに分けて、重みを更新する

・**epochs** : 学習の繰り返し。60000個を何回学習するか

・**validation_split** : 学習データの何パーセントをvalidation用データにするか(各エポックのテスト。

**コールバック関数**

keras.callbacks内の関数を使ってモデルの保存ができる。

・**check = ModelCheckpoint("modelname.hdf5")**で、学習済みモデルの重みを保存する

その場合、model.fit(hogehoge,callbacks=[check])としてやる

・tensorboardなどとも連携可能

・自分でcallbackしたい関数を作ることも可能

In [13]:
from keras.callbacks import ModelCheckpoint

check = ModelCheckpoint('model.hdf5')

history = model.fit(X_train, y_train, epochs=20,
                   validation_split=0.2, batch_size=32,
                   callbacks=[check])

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 9s 188us/step - loss: 0.3583 - acc: 0.8957 - val_loss: 0.2386 - val_acc: 0.9326
Epoch 2/20
48000/48000 [==============================] - 8s 170us/step - loss: 0.2179 - acc: 0.9353 - val_loss: 0.1788 - val_acc: 0.9506
Epoch 3/20
48000/48000 [==============================] - 8s 169us/step - loss: 0.1609 - acc: 0.9528 - val_loss: 0.1533 - val_acc: 0.9539
Epoch 4/20
48000/48000 [==============================] - 8s 171us/step - loss: 0.1286 - acc: 0.9616 - val_loss: 0.1257 - val_acc: 0.9648
Epoch 5/20
48000/48000 [==============================] - 8s 170us/step - loss: 0.1082 - acc: 0.9674 - val_loss: 0.1176 - val_acc: 0.9653
Epoch 6/20
48000/48000 [==============================] - 8s 168us/step - loss: 0.0932 - acc: 0.9729 - val_loss: 0.1078 - val_acc: 0.9677
Epoch 7/20
48000/48000 [==============================] - 8s 170us/step - loss: 0.0819 - acc: 0.9762 - val_loss: 0.1033 - 

### モデルの評価 

model.evaluate()という関数で、テストデータを用いたモデルの評価が可能。lossとaccuracyを見ている

In [14]:
loss, accuracy = model.evaluate(X_test, y_test)
print('\nloss:{} accuracy:{}'.format(loss, accuracy))

10000/10000 [==============================] - 1s 66us/step

loss:0.09906537595200353 accuracy:0.9726
